In [18]:
from metaflow import Metaflow, Flow, Run, Step, Task, DataArtifact

In [19]:
Metaflow().flows

[Flow('ModelFlow')]

In [20]:
flow = Metaflow()['ModelFlow']


In [21]:
import pandas as pd
from datetime import datetime, date, time, timedelta

def to_datetime(str_date):
    type(str_date)
    if(str_date != None):
        str_date = str_date.replace('T', ' ').replace('.%fZ', '')
        return datetime.strptime(str_date, '%Y-%m-%d %H:%M:%S')
    return None

def duration(init_date, end_date):
    if(end_date != None):
        delta = end_date - init_date
        return delta.total_seconds() / 60 / 60
    return -1



In [22]:
flow_name = 'ModelFlow'
flow = Metaflow()[flow_name]
flow_runs = {}
columns = ['run_id', 'version_name', 'created_at', 'finished_at', 'run_duration', 'scores_mean', 'scores_std']

for idx, run in enumerate(list(flow)):
    if(run.finished):
        scores = run['build_model'].task.data.scores.T
        scores_mean= scores.values[0].mean()
        scores_std = scores.values[0].std()

        version_name = run['start'].task.data.version_name
        created_at = to_datetime(run.created_at)
        finished_at = to_datetime(run.finished_at)
        run_duration = duration(created_at, finished_at)

        values = [run.id, version_name, created_at, finished_at, run_duration, scores_mean, scores_std]
        flow_runs[idx] = values
    
flow_runs = pd.DataFrame.from_dict(flow_runs, orient='index', columns = columns)   
flow_runs

,run_id,version_name,created_at,finished_at,run_duration,scores_mean,scores_std
0,1581704415059749,sample_100%,2020-02-14 18:20:15,2020-02-14 20:31:46,2.191944,0.932840,0.059364
2,1581628145437015,sample_10%,2020-02-13 21:09:05,2020-02-13 21:28:24,0.321944,0.917974,0.075777
5,1581624955229034,sample_1000,2020-02-13 20:15:55,2020-02-13 20:17:37,0.028333,0.896741,0.153555
6,1581613732312297,sample_100,2020-02-13 17:08:52,2020-02-13 17:10:36,0.028889,0.882982,0.144704


In [23]:
Run('ModelFlow/1581625433004933').finished

False